## Máster en Data Science

### Machine Learning

Contacto: angel.blanco@cunef.edu

<div style="text-align: justify;">

# **Demo de producción**

En este notebook, se lleva a cabo la ejecución de los preprocesadores para guardar los parámetros necesarios para realizar predicciones en el servidor, así como la explicación del proceso seguido para crear la API.

La idea es que, cuando lleguen datos nuevos en producción, el programa sea capaz de gestionar variables extrañas, imputar valores faltantes en los datos, escalarlos y hacerles encoding para que el modelo funcione correctamente.

Para las variables categóricas, se ha utilizado la moda para sustituir valores en los casos en los que los datos estén incompletos, con el fin de utilizar el valor que más representado aparece.

Por otro lado, para las variables numéricas, se ha calculado la mediana de cada una únicamente con los valores no omitidos del dataset original. 

En cuanto al encoding y escalado de variables, se han utilizado los métodos de One Hot Encoding y Standard Scaler.

El porqué de la elección de estas técnicas y sus beneficios se explican más abajo en el documento.

</div>

### Librerías

In [1]:
import os
from pathlib import Path

# Cambio el directory al root del proyecto
current_dir = Path.cwd()

if current_dir.name == "notebooks":
    os.chdir(current_dir.parent)

# Procesado
import json
import pandas as pd

# Encoding y escalado
from sklearn.preprocessing import StandardScaler
import category_encoders as ce

# Funciones y jsons
from src.const import variable_types, model_variables
from src.models import write_model
from src.data import read_csv

### Carga de datos

Primero, se cargan los datos tratados del entrenamiento y test.

In [2]:
data = pd.read_csv("data/raw/Base.csv")
pd_fraud_train = read_csv("data/preprocessed/train_pd_data_preprocessing_missing_outlier.csv")
pd_fraud_test = read_csv("data/preprocessed/test_pd_data_preprocessing_missing_outlier.csv")

### Selección de variables

Se debe descartar la variable objetivo porque no se sabe su valor con los nuevos datos que entren en la demo. Es precisamente eso lo que queremos predecir.

In [3]:
# Drop de la variable objetivo porque no se sabrá en producción
x_train = pd_fraud_train.drop('fraud_bool',axis=1)
x_test = pd_fraud_test.drop('fraud_bool',axis=1)
y_train = pd_fraud_train['fraud_bool']
y_test = pd_fraud_test['fraud_bool']

Se seleccionan las variables cuyo nombre coincide con las que se han utilizado en el modelo antes de ser codificadas y escaladas, ignorando las variables desconocidas.

In [4]:
# Selección de las variables que se han usado en el modelo
x_train = x_train[model_variables["raw"]]
x_test = x_test[model_variables["raw"]] 

### Clasificación de variables

In [5]:
# Clasificación por tipo
numerical_vars = [v for v in variable_types["numericals"] if v in model_variables["raw"]]
categorical_vars = [v for v in variable_types["categoricals"] if v in model_variables["raw"]]
binary_vars = [v for v in variable_types["binary"] if v in model_variables["raw"]]

x_train_cat = x_train[categorical_vars]
x_test_cat = x_test[categorical_vars]

x_train_binary = x_train[binary_vars]
x_test_binary = x_test[binary_vars]

x_train_numerical = x_train[numerical_vars]
x_test_numerical = x_test[numerical_vars]

### Reemplazo de valores NA

In [6]:
# Cálculo de los valores que serán utilizados para reemplazar valores faltantes en producción

# Uso de la moda para variables categóricas para representar el valor más frecuente
modes = x_train_cat.mode().T.to_dict()[0]

# Uno de la mediana para numéricas para representar la tendencia 
medians = {}
for column in x_train_numerical.columns:

    # Mediana calculada solo para valores positivos
    median = (
        data
            .dropna(subset=column)
            .query(f"{column} >= 0")[column]
            .median()
        )
    
    # Guardado de resultados en el diccionario vacío
    medians[column] = median

A continuación, se rellenan los valores NA según el tipo de variable y se guardan esos valores en un diccionario en json para poder reutilizarlos con los datos nuevos que reciba la demo. De esta manera, el modelo sabrá qué hacer con los valores faltantes de los datasets introducidos y también evitará que surjan problemas posteriores en la codificación de variables.

In [7]:
fill_na_values = medians | modes
with open("metadata/fill_na_values.json", "w") as f:
    json.dump(fill_na_values, f)

Como las variables de los datos nuevos que se reciben están sin escalar ni codificar, deben ser correctamente aplicados ambos procesos de manera que las variables con datos nuevos, coincidan con las variables con las que se ha entrenado el modelo y así pueda funcionar correctamente.

<div style="text-align: justify;">

### Fit y guardado del one hot encoder

Cuando se aplica one-hot encoding a una variable categórica, cada categoría se representa como un vector binario único. En lugar de asignar un número entero a cada categoría, se crea un vector con la longitud igual al número total de categorías, y todos los elementos del vector son establecidos en cero, excepto el correspondiente a la categoría que se está representando, que se establece en uno.

La principal ventaja del este método es que permite a los algoritmos de machine learning trabajar de manera efectiva con variables categóricas, ya que las transforma en una forma numérica sin introducir un orden o jerarquía artificial entre las categorías (como sí lo harían métodos como la codificación ordinal). Esto es particularmente útil en situaciones donde las categorías no tienen un orden inherente, como en este caso, nuestra variable del tipo de sistema operativo.

Además, evita problemas de interpretación errónea de las relaciones entre las categorías debido a los números asignados. Esta interpretación errónea podría afectar negativamente el rendimiento del modelo, ya que introduce en él información incorrecta.

Uno de los inconvenientes de esta técnica, es que aumenta considerablemente la dimensión del conjunto de datos para el modelo, lo que podría afectar negativamente a su rendimiento computacional. En este caso, como no hay muchas variables categóricas y las que hay no tienen muchas clases diferentes, esto en principio no sería un problema.

</div>

In [8]:
# Fit del one hot encoder
hot = ce.OneHotEncoder(cols=variable_types['categoricals'])
one_hot_encoder = hot.fit(x_train_cat, y_train)

# Guardado del encoder
write_model(one_hot_encoder, "one_hot_encoder.pkl")

<div style="text-align: justify;">

### Fit y guardado del escalador

StandardScaler es una técnica de normalización que transforma los datos de una variable para que tengan una media de cero y una desviación estándar de uno. Esta transformación es especialmente útil cuando las variables de entrada de un modelo de aprendizaje automático tienen escalas diferentes, como en este caso.

La regresión logística utiliza técnicas de optimización, como el descenso de gradiente, para encontrar los valores óptimos de los coeficientes del modelo. Si las variables de entrada tienen escalas muy diferentes, el descenso de gradiente puede converger más lentamente o incluso no converger en absoluto. Esto se debe a que las actualizaciones de los coeficientes estarán dominadas por las variables con magnitudes más grandes, haciendo que las variables con magnitudes más pequeñas tengan un menor impacto en el proceso de optimización.

Por esta razón, es una buena práctica escalar las variables antes de aplicar la regresión logística. Al utilizar técnicas de escalado, todas las variables tendrán una magnitud comparable, lo que facilita la convergencia del algoritmo de optimización y mejora el rendimiento general del modelo.

</div>

In [9]:
# Fit del scaler
scaler = StandardScaler()
model_scaled = scaler.fit(x_train_numerical)

# Guardado del scaler
write_model(scaler, "scaler")

Tanto el encoder como el escalador han sido guardados por el mismo motivo, para reutilizarlos en producción.

<div style="text-align: justify;">

# **Explicacion de Docker y el server del modelo y dashboard**

Una vez que se ha llevado a cabo este proceso, se continúa con la construcción del server y el dashboard, y finalmente su implementación en una imagen de Docker. A continuación, una explicación sobre el server, el dashboard, su implementación en Docker y lo que supone la adición de la herramienta Compose al contenedor.

## **Server**

El server ha sido construido con Flask en el archivo app.py. Este archivo contiene el modelo de regresión logística para predecir la probabilidad de fraude en datos proporcionados. Comienza con la creación de una instancia de Flask llamada app. Con esto se nombra al sevidor:

```python
app = Flask(__name__)
```



A continuación se cargan los modelos y procesadores. Se han utilizado el modelo entrenado de regresión logística, el one hot encoder y el escaler guardados:

  ```python
  model = read_model("LogisticRegression")
  one_hot_encoder = read_model("one_hot_encoder")
  scaler = read_model("scaler")
  ```

Tras esto, se definen las rutas del servidor. Se definen dos rutas: una para la raíz ("/") que simplemente imprime las cabeceras de la solicitud y devuelve un mensaje indicando que el servidor está en funcionamiento, y otra ("/predict") que se utiliza para realizar predicciones basadas en datos proporcionados mediante una solicitud POST de Flask:

```python
@app.route("/")

def hello_world():
    print(request.headers)

    return "Server is running"

@app.route("/predict", methods=["POST"])

def predict():

    # Verifición de que los datos del cliente han sido recibidos correstamente por el servidor
    print(f"REQUEST: {request.json}")

    # Conversión del json a un dataframe para sklearn
    data = pd.DataFrame(request.json["data"])

    # Filtrado de las variables raw en caso de que haya una variable extraña
    for column in data.columns:
        
        # Drop de columnas que no coinciden con las que están en el modelo
        if column not in model_variables["raw"]:
            data = data.drop(column, axis=1)
    
    # Comprobación de variables faltantes
    missing_variables = [var for var in model_variables["raw"] if var not in data.columns]

    # Devolución de error si falta alguna variable
    if missing_variables:
        return {
            "message": f"Some of the required variables are missing: {missing_variables}"
        }, 400

    # Preprocesamiento de los datos recibidos
    # Aquí es donde se usan los métodos guardados anteriormente
    X = preprocess(
        data=data,
        fill_na_values=fill_na_values,
        one_hot_encoder=one_hot_encoder,
        scaler=scaler,
        variable_types=variable_types,
        model_variables=model_variables["raw"]
    )

    # Especificación de que solo se usen las variables vistas durante el fit para evitar errores
    X = X[model.feature_names_in_]

    # Guardado de predicciones y probabilidades
    prediction = [int(p) for p in model.predict(X)]
    probas = model.predict_proba(X)
    fraud_proba = [float(round(p, 4)) for p in probas[0:,1]]

    # Construcción de un diccionario con ambas
    response = {
        "prediction": prediction,
        "probability": fraud_proba,
    }

    # Devolución de la respuesta en la consola
    pprint(response)

    return response
```

<div style="text-align: justify;">

En resumen, la función predict toma datos proporcionados en formato JSON, realiza algunas verificaciones y preprocesamientos, y luego utiliza el modelo cargado para hacer predicciones.

Y, finalmente, se se comprueba si el script se está ejecutando directamente y no como un módulo importado. Si es así, se ejecuta la aplicación Flask en el puerto 5000 en modo debug:

```python
if __name__ == "__main__":
    app.run(port=5000, debug=True)
```

Al ejecutar este script, se puede acceder al servidor a través de la ruta localhost:5000 y ver el mensaje "Server is running". Para realizar predicciones, se utilizaría la ruta localhost:5000/predict mediante solicitudes POST con datos JSON. El comando para arrancar el servidor es:

> Nota:  
> Para este proyecto se ha utilizado Bash, por lo que los comandos de la consola que aparecen referenciados son para este intérprete.

<div align="center">

<br>

```bash
python3 app.py run
```
</div>

<br>

Debería verse tal que así:

<br>

<img src='../images/server_running.png' style="width:600px; display: block; margin: auto;">

<br>

## **Dashboard**

Una vez que se ha construído el server, el siguiente paso es la construcción del dashboard que servirá de interfaz para interactuar con el modelo sin necesidad de que el usuario tenga que programar. Este proceso se ha realizado en el archivo dashboard.py.

<br>
<div align="center">

<span style="font-size: 1.5em;">[**Dashboard.py explicado**](../dashboard.py)

</div>
<br>
La construcción se ha realizado de manera que, en el dashboard, aparezcan dos modos diferentes de usar la API, uno subiendo un csv o Excel con datos, y otro interactuando manualmente con la interfaz e introduciendo valores en la barra lateral. Se añadió un logotipo por estética y una serie de anotaciones para guiar al usuario en el uso. Para acceder al dashboard, una vez que el servidor está activo, se utiliza este comando:  

<br>
<br>
<div align="center">

```BASH
streamlit run dashboard.py
```

</div>
<br>

Y se encuentra en [localhost:8501](http://localhost:8501/) (ruta predeterminada). La interfaz deberá verse ahí.


### ¿Cómo se utiliza la API?

A continuación un pequeño walkthrough de los modos que tiene y como se utilizan:  

<div align="center">

<span style="font-size: 1.5em;">[**Instrucciones**](../docs/instructions.md)

</div>

## **Docker**

Docker es una plataforma de virtualización a nivel de contenedores que simplifica el desarrollo, despliegue y ejecución de aplicaciones al encapsularlas junto con sus dependencias en contenedores portátiles. 

El proceso a seguir para usar docker es el que describe la imagen mostrada a continuación:

<br>

<img src='../images/docker_process_esquema.png' style="width:600px; display: block; margin: auto;">

<br>

Para entender este proceso, hay que entender primero unos conceptos básicos de docker.

Un **dockerfile** es un archivo de texto plano que contiene una serie de instrucciones que le indican a Docker cómo construir una imagen de contenedor. Las imágenes de Docker son la base para la ejecución de contenedores, y un Dockerfile es esencial para crear estas imágenes de manera reproducible y automatizada. Cada instrucción en un Dockerfile crea una nueva capa en la imagen resultante. En este caso, hay dos dockerfiles, uno para el servidor y otro para el dashboard de streamlit, cada uno con sus instrucciones correspondientes.

Una **imagen de Docker** es un paquete ligero, portátil y que incluye todo lo necesario para ejecutar una aplicación, incluyendo el código, las bibliotecas, las dependencias y las configuraciones del entorno. Las imágenes se construyen como imágenes de otras, por ejemplo, en este caso se ha usado _python:3.10-slim-bullseye_ para construir las del proyecto. 

Estas imágenes se utilizan como plantillas para crear contenedores. Cada contenedor se crea a partir de una única imagen, y esa imagen determina el entorno y los componentes que estarán presentes en el contenedor. Se utiliza el comando _"docker build"_ para construir la imagen a partir del Dockerfile. En este caso:
  
<div align="center">

  ```BASH
     docker build -t angelbg34/model-server .
     
     docker build -t angelbg34/dashboard .
  ```

</div>

Esto construye las imágenes, que se almacenan localmente.

Los **contenedores** son instancias individuales y aisladas de una imagen específica, que proporcionan aislamiento y portabilidad. Después de construir la imagen, puede ejecutarse un contenedor local para asegurarse de que todo funciona como se espera. Lo que permite realizar pruebas y depuraciones antes de compartir la imagen. En este caso:

<div align="center">

  ```BASH
     docker run --name angelbg34/model-server:latest
     
     docker run --name angelbg34/dashboard:latest
  ```

</div>

> Nota:   
> El uso de --name es opcional, si no se pone, Docker asigna un nombre aletorio a los contenedores. La etiqueta latest indica que la versión que se va a ejecutar es la más reciente.

**Docker Hub** es un servicio en la nube que actúa como un registro público y privado de imágenes de contenedores. Este servicio permite a los desarrolladores, equipos y organizaciones almacenar, compartir y distribuir imágenes de contenedores de manera eficiente. Una vez creadas las imágenes, probadas en local hasta que queden como se desea y habiendo hecho login en la cuenta de Docker a la que se desean subir las imágenes, se suben a Docker Hub con _docker push_. En este caso: 

<br>
<div align="center">

```BASH 
    docker push angelbg34/model-server:latest

    docker push angelbg34/dashboard:latest
```
</div>
<br>

Una vez que las imágenes están subidas, las personas interesadas en utilizarlas pueden usar el comando _docker pull_ para descargarlas en local, o el comando _docker run_ para ejecutarlas. En este caso:

<br>
<div align="center">

```BASH
    docker pull angelbg34/server:latest

    docker pull angelbg34/dashboard:latest
```
<br>

```BASH
    docker run angelbg34/model-server:latest

    docker run angelbg34/dashboard:latest
```
</div>

> Nota:  
> Al hacer docker run, si la imagen no está disponible en local, se descarga automáticamente en segundo plano antes de iniciarse.

<br>

### ¿Por qué usar Docker?
  
  Porque permite encapsular todas las dependencias, bibliotecas y configuraciones necesarias para la API en un contenedor. Esto evita conflictos y asegura que se tenga acceso a las versiones específicas de las bibliotecas que necesita, sin afectar el entorno del host.

<br>

<img src='../images/docker_works.png' style="width:600px; display: block; margin: auto;">

<br>
  
  Las diferencias entre entornos locales y servidores de producción pueden causar problemas inesperados. Docker garantiza que la aplicación se ejecute de la misma manera en cualquier lugar por su aislamiento de dependencias.

  La capacidad de definir la infraestructura como código en archivos Dockerfile y Docker Compose, facilita el mantenimiento. Actualizar la aplicación o cambiar la configuración se hace de manera reproducible y controlada, reduciendo los riesgos asociados a cambios no planificados. También facilita la reversión a versiones anteriores. Si algo sale mal después de un despliegue, puede volverse rápidamente a una versión anterior sin afectar a la integridad del sistema.

  Los contenedores son independientes del entorno subyacente. Se puede construir y probar aplicaciones en local y estar seguro de que funcionarán de la misma forma en cualquier entorno que ejecute Docker.

<br>

<img src='../images/docker_works_everywhere.png' style="width:600px; display: block; margin: auto;">
  
<br>

  Docker tiene un Hub que es un registro público donde puedes compartir imágenes. Esto facilita la colaboración y la distribución de la API, ya que otros desarrolladores pueden implementarla con facilidad utilizando la misma imagen.

### Docker Compose
  
  Docker Compose es una herramienta que permite definir y gestionar aplicaciones Docker multi-contenedor mediante un archivo de configuración YAML (archivo "docker-compose.yml" en este caso). Este archivo describe los servicios, redes y volúmenes necesarios para la aplicación, así como las configuraciones específicas de cada servicio, como las imágenes Docker a utilizar, los puertos a exponer y las dependencias entre servicios.


### Preview del archivo explicado:

  ```YAML
  # Desglose de los archivos necesarios para construir el contenedor 
  services:
    
    # Dockerfile.server es el archivo que construirá el server
    # contiene la configuración para desencadenar el copiado 
    # y la puesta en marcha de los archivos necesarios
    # angelbg34/model-server:latest es la imagen utilizada para el server
    model-server:
      image: angelbg34/model-server:latest
      container_name: model-server
      build: 
        context: .
        dockerfile: docker/Dockerfile.server
  
      # El puerto 5000 es el puerto predeterminado de Flask
      ports:
        - 5000:5000
  
    # El dashboard es la app de la librería streamlit usada para interactuar con el modelo
    # Dockerfile.dashboard es el archivo que construirá el dashboard
    # angelbg34/dashboard:latest es la imagen utilizada para el dashboard
    dashboard:
      image: angelbg34/dashboard:latest
      container_name: dashboard
      build: 
        context: .
        dockerfile: docker/Dockerfile.dashboard
  
      # El puerto 8501 es el predeterminado de streamlit
      ports:
        - 8501:8501
      
      # Unión del archivo de configuración del enviroment al container
      env_file:
        - .env
  
      # Espera a que model-server esté corriendo para activarse el proceso
      depends_on:
        model-server: 
          condition: service_started 
  ```

### Beneficios de utilizar Docker Compose:

Al añadir un compose al contenedor, se simplifica la configuración de la aplicación porque se define en un único archivo YAML. Esto facilita la comprensión y gestión de los servicios, redes y volúmenes necesarios, reduciendo la complejidad en comparación con la configuración manual de cada contenedor. 

También, compose permite definir y orquestar los múltiples servicios que componen una aplicación (en este caso son solamente dos los principales, pero en la realidad se utilizan muchos más). En el archivo se especifica la relación y dependencia entre model-server y dashboard, lo que simplifica la gestión de los componentes de la API.

Como se define toda la configuración de la aplicación en un solo archivo, asegura un despliegue exitoso en enviroments diferentes. Esto es fundamental para mantener la consistencia entre el enviroment de desarrollo, testeo y producción, ya que se quiere que la API funcione en cualquier ordenador.

Por último, es más fácil de usar en consola. Con un simple conjunto de comandos, puede construirse, desplegarse y gestionarse la API.

<br>

### Proceso con Docker Compose:

Durante el desarrollo del proyecto se utiliza _docker compose build_ pero para utilizar el servicio no hace falta porque las imágenes ya están disponibles en Docker Hub. Gracias al compose, lo único que habría que hacer sería:

<br>

1. Descarga de las imágenes de Docker:

<div align="center">

   ```BASH
   docker compose pull
   ```

</div>

2. Puesta en marcha del contenedor:

<div align="center">

   ```BASH
   docker compose up
   ```

</div>

</div>